In [28]:
import sys
import os

In [29]:
sys.path.append(os.path.abspath('../../../../src'))

from read_and_write_docs import read_jsonl, read_rds, write_jsonl
from tokenize_and_score import load_model
from utils import apply_temp_doc_id, build_metadata_df
from n_gram_functions import common_ngrams, pretty_print_common_ngrams

In [30]:
corpus = "Wiki"
data_type = "training"

known_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/{corpus}/known_raw.jsonl"
known = read_jsonl(known_loc)
known = apply_temp_doc_id(known)

unknown_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/{corpus}/unknown_raw.jsonl"
unknown = read_jsonl(unknown_loc)
unknown_df = apply_temp_doc_id(unknown)

metadata_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/metadata.rds"
metadata = read_rds(metadata_loc)
filtered_metadata = metadata[metadata['corpus'] == corpus]
agg_metadata = build_metadata_df(filtered_metadata, known, unknown)

In [31]:
problem_dataset_agg = read_jsonl("/Users/user/Documents/test_data/n-gram_tracing/Wiki_training_agg.jsonl")
problem_dataset_profile = read_jsonl("/Users/user/Documents/test_data/n-gram_tracing/Wiki_training_profile.jsonl")

In [40]:
diff_author_problems = problem_dataset_agg[problem_dataset_agg['known_author'] != problem_dataset_agg['unknown_author']].copy()
diff_author_problems.sort_values(["highest_common_count"], ascending=[False], inplace=True)
diff_author_problems[diff_author_problems['highest_common_count'] >= 3].head(20)

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
287,Enemesis vs Equanimous1,Enemesis,Equanimous1,enemesis_text_3,equanimous1_text_5,5,Ġthe Ġcurrent Ġstate Ġof Ġthe
161,Caboga vs Chanakyathegreat,Caboga,Chanakyathegreat,caboga_text_5,chanakyathegreat_text_1,5,Ġ 2 0 0 8
262,Dweller vs Ecelan,Dweller,Ecelan,dweller_text_3,ecelan_text_2,5,.Ċ i Ġdon 't Ġthink
364,Fyunck(click) vs Garda40,Fyunck(click),Garda40,fyunck_click_text_12,garda40_text_1,5,Ġ 2 0 1 1
365,Fyunck(click) vs Garda40,Fyunck(click),Garda40,fyunck_click_text_2,garda40_text_1,5,Ġ 2 0 1 2
395,Gwen_Gale vs Habap,Gwen_Gale,Habap,gwen_gale_text_3,habap_text_1,5,", Ġbut Ġi Ġdon 't"
286,Enemesis vs Equanimous1,Enemesis,Equanimous1,enemesis_text_2,equanimous1_text_5,5,Ġthe Ġcurrent Ġstate Ġof Ġthe
441,HeadleyDown vs Hipocrite,HeadleyDown,Hipocrite,headleydown_text_2,hipocrite_text_4,5,Ġon Ġthis Ġtalk Ġpage .Ċ
135,Britmax vs BrotherDarksoul,Britmax,BrotherDarksoul,britmax_text_1,brotherdarksoul_text_4,4,Ġis Ġno Ġneed Ġto
165,Chanakyathegreat vs Cla68,Chanakyathegreat,Cla68,chanakyathegreat_text_10,cla68_text_1,4,", Ġi Ġdon 't"


In [41]:
known_doc = "enemesis_text_2"
known_text = known[known['doc_id'] == known_doc].reset_index().loc[0, 'text']

unknown_doc = "equanimous1_text_5"
unknown_text = unknown[unknown['doc_id'] == unknown_doc].reset_index().loc[0, 'text']

In [34]:
tokenizer, model = load_model("/Volumes/BCross/models/Qwen 2.5/Qwen2.5-0.5B-Instruct")

In [42]:
common = common_ngrams(known_text, unknown_text, 2, model, tokenizer)

In [43]:
pretty_print_common_ngrams(common, tokenizer=tokenizer, order='len_asc')

2-grams (15): [' an agenda', ' an article', ' from the', ' in a', ' in the', ' information is', ' is not', ' this article', ' to be', ' to the', ' which is', ' you can', ' you have', '.\nthe', '.\nthey']
3-grams (2): [' the article and', '.\nthere is']
5-grams (1): [' the current state of the']


In [15]:
print(f"""
<DOC>
{known_text}
</DOC>
<NGRAM>
""")


<DOC>
We simply can not permit further stonewalling via tactics such as avoiding high-profile efforts at If the eight editors you point to and allege harbor views that are contrary to the consensus truly had felt they had a meritorious argument, they should have participated in the poll.
We simply can not permit further stonewalling via tactics such avoiding high-profile efforts at Your argument is nonsense and merits no further response than this The wording is not irrelevant to the 17 people who participated in the poll.
The consensus view is against your reasoning, as well as your wishes.
You could have voted this, that, or something else.
Your argument is nonsense and merits no further response than this You could have voted this, that, or something else.
Try reading Wikipedia s Whether or not you agree with the 17 people who participated in the poll and the new numbers reinforcing that message in Dicklyon s, you will have to respect it.
TheFreeloader, there was a minority crew of

## Phrase 1 -  " allege harbor views that are contrary to the consensus"

In [18]:
p_1 = " allege harbor views that are contrary to the consensus"

para_1 = [ " allege that they harbor views that are contrary to the consensus", " allege that they harbor views that are opposed to the consensus", " allege that they harbor views that are at odds with the consensus", " allege that they harbor opinions that are contrary to the consensus", " allege that they harbor opinions that are opposed to the consensus", " allege that they harbor opinions that are at odds with the consensus", " allege that they hold views that are contrary to the consensus", " allege that they hold views that are opposed to the consensus", " allege that they hold views that are at odds with the consensus", " allege that they hold opinions that are contrary to the consensus", " allege that they hold opinions that are opposed to the consensus", " allege that they hold opinions that are at odds with the consensus", " allege that they entertain views that are contrary to the consensus", " allege that they entertain views that are opposed to the consensus", " allege that they entertain views that are at odds with the consensus", " allege that they entertain opinions that are contrary to the consensus", " allege that they entertain opinions that are opposed to the consensus", " allege that they entertain opinions that are at odds with the consensus", " claim that they harbor views that are contrary to the consensus", " claim that they harbor views that are opposed to the consensus", " claim that they harbor views that are at odds with the consensus", " claim that they harbor opinions that are contrary to the consensus", " claim that they harbor opinions that are opposed to the consensus", " claim that they harbor opinions that are at odds with the consensus", " claim that they hold views that are contrary to the consensus", " claim that they hold views that are opposed to the consensus", " claim that they hold views that are at odds with the consensus", " claim that they hold opinions that are contrary to the consensus", " claim that they hold opinions that are opposed to the consensus", " claim that they hold opinions that are at odds with the consensus", " claim that they entertain views that are contrary to the consensus", " claim that they entertain views that are opposed to the consensus", " claim that they entertain views that are at odds with the consensus", " claim that they entertain opinions that are contrary to the consensus", " claim that they entertain opinions that are opposed to the consensus", " claim that they entertain opinions that are at odds with the consensus", " assert that they harbor views that are contrary to the consensus", " assert that they harbor views that are opposed to the consensus", " assert that they harbor views that are at odds with the consensus", " assert that they harbor opinions that are contrary to the consensus", " assert that they harbor opinions that are opposed to the consensus", " assert that they harbor opinions that are at odds with the consensus", " assert that they hold views that are contrary to the consensus", " assert that they hold views that are opposed to the consensus", " assert that they hold views that are at odds with the consensus", " assert that they hold opinions that are contrary to the consensus", " assert that they hold opinions that are opposed to the consensus", " assert that they hold opinions that are at odds with the consensus", " assert that they entertain views that are contrary to the consensus", " assert that they entertain views that are opposed to the consensus", " assert that they entertain views that are at odds with the consensus", " assert that they entertain opinions that are contrary to the consensus", " assert that they entertain opinions that are opposed to the consensus", " assert that they entertain opinions that are at odds with the consensus", " contend that they harbor views that are contrary to the consensus", " contend that they harbor views that are opposed to the consensus", " contend that they harbor views that are at odds with the consensus", " contend that they harbor opinions that are contrary to the consensus", " contend that they harbor opinions that are opposed to the consensus", " contend that they harbor opinions that are at odds with the consensus", " contend that they hold views that are contrary to the consensus", " contend that they hold views that are opposed to the consensus", " contend that they hold views that are at odds with the consensus", " contend that they hold opinions that are contrary to the consensus", " contend that they hold opinions that are opposed to the consensus", " contend that they hold opinions that are at odds with the consensus", " contend that they entertain views that are contrary to the consensus", " contend that they entertain views that are opposed to the consensus", " contend that they entertain views that are at odds with the consensus", " contend that they entertain opinions that are contrary to the consensus", " contend that they entertain opinions that are opposed to the consensus", " contend that they entertain opinions that are at odds with the consensus", " allege that they harbour views that are contrary to the consensus", " allege that they harbour views that are opposed to the consensus", " allege that they harbour views that are at odds with the consensus", " allege that they harbour opinions that are contrary to the consensus", " allege that they harbour opinions that are opposed to the consensus", " allege that they harbour opinions that are at odds with the consensus", " assert that they harbour views that are contrary to the consensus", " assert that they harbour views that are opposed to the consensus", " assert that they harbour views that are at odds with the consensus", " assert that they harbour opinions that are contrary to the consensus", " assert that they harbour opinions that are opposed to the consensus", " assert that they harbour opinions that are at odds with the consensus", " allege that they have views that are contrary to the consensus", " allege that they have views that are at odds with the consensus", " claim that they have views that are contrary to the consensus", " claim that they have views that are at odds with the consensus", " assert that they have views that are contrary to the consensus", " assert that they have views that are at odds with the consensus", " allege that they hold positions that are contrary to the consensus", " allege that they hold positions that are opposed to the consensus", " allege that they hold positions that are at odds with the consensus", " claim that they hold positions that are contrary to the consensus", " claim that they hold positions that are opposed to the consensus", " claim that they hold positions that are at odds with the consensus", " allege that they harbor views that are against the consensus", " allege that they harbor views that are in conflict with the consensus", " allege that they harbor views that are inconsistent with the consensus", " allege that they harbor views that are counter to the consensus", " make the allegation that they hold views that are contrary to the consensus", " make the allegation that they hold views that are opposed to the consensus", " make the allegation that they hold views that are at odds with the consensus", " make the claim that they hold views that are contrary to the consensus", " make the claim that they hold views that are opposed to the consensus", " make the claim that they hold views that are at odds with the consensus", " make the assertion that they hold views that are contrary to the consensus", " make the assertion that they hold views that are opposed to the consensus", " make the assertion that they hold views that are at odds with the consensus", " maintain that they harbor beliefs that are contrary to the consensus", " maintain that they harbor beliefs that are opposed to the consensus", " maintain that they harbor beliefs that are at odds with the consensus", " aver that they harbor beliefs that are contrary to the consensus", " aver that they harbor beliefs that are opposed to the consensus", " aver that they harbor beliefs that are at odds with the consensus", " allege they harbor views that are contrary to the consensus", " allege they harbor views that are opposed to the consensus", " allege they harbor views that are at odds with the consensus", " claim they harbor views that are contrary to the consensus", " claim they harbor views that are opposed to the consensus", " claim they harbor views that are at odds with the consensus", " assert they harbor views that are contrary to the consensus", " assert they harbor views that are opposed to the consensus", " assert they harbor views that are at odds with the consensus", " contend they hold opinions that are contrary to the consensus", " contend they hold opinions that are at odds with the consensus", " assert that they entertain beliefs that are contrary to the consensus", " assert that they entertain beliefs that are at odds with the consensus", " allege that they harbour beliefs that are contrary to the consensus", " allege that they harbour beliefs that are at odds with the consensus", " maintain they hold views that are contrary to the consensus", " maintain they hold views that are at odds with the consensus", " aver they hold views that are contrary to the consensus", " aver they hold views that are at odds with the consensus", " claim that they harbour opinions that are against the consensus" ]

## Phrase 2 - ", they should have participated in the poll.\n"

In [19]:
p_2 = ", they should have participated in the poll.\n"

para_2 = [", they should've participated in the poll.\n", ", they should have taken part in the poll.\n", ", they should've taken part in the poll.\n", ", they should have joined in the poll.\n", ", they should've joined in the poll.\n", ", they should have joined the poll.\n", ", they should've joined the poll.\n", ", they should have been part of the poll.\n", ", they should've been part of the poll.\n", ", they should have taken part in the polling.\n", ", they should've taken part in the polling.\n", ", they should have participated in the polling.\n", ", they should've participated in the polling.\n", ", they should have engaged in the poll.\n", ", they should've engaged in the poll.\n", ", they should have partaken in the poll.\n", ", they should've partaken in the poll.\n", ", in the poll they should have participated.\n", ", in the poll they should've participated.\n", ", the poll should have been participated in by them.\n", ", the poll should've been participated in by them.\n"]

## Phrase 3 - " had felt they had"

In [ ]:
p_3 = " had felt they had"

para_3 = [" had felt they'd", " had felt that they had", " had felt that they'd", " had believed they had", " had believed they'd", " had believed that they had", " had believed that they'd", " had thought they had", " had thought they'd", " had thought that they had", " had thought that they'd", " had supposed they had", " had supposed they'd", " had supposed that they had", " had supposed that they'd", " had reckoned they had", " had reckoned they'd", " had reckoned that they had", " had reckoned that they'd", " had sensed they had", " had sensed they'd", " had sensed that they had", " had sensed that they'd", " had perceived they had", " had perceived they'd", " had perceived that they had", " had perceived that they'd", " had suspected they had", " had suspected they'd", " had suspected that they had", " had suspected that they'd", " had assumed they had", " had assumed they'd", " had assumed that they had", " had assumed that they'd", " had had a feeling they had", " had had a feeling they'd", " had had the feeling they had", " had had the feeling they'd", " had felt they possessed", " had felt that they possessed", " had believed they possessed", " had thought they possessed", " had supposed they possessed", " had reckoned they possessed", " had sensed they possessed", " had perceived they possessed", " had suspected they possessed", " had assumed they possessed", " had had a sense they had", " had had the sense they had", " had had a sense that they had", " had had the sense that they had", " had opined they had", " had opined that they had", " had concluded they had", " had concluded that they had"]
